## Imports

In [1]:
import pandas as pd
import numpy as np
import datetime, random

from multiprocessing import Pool
import time

#Potentially useful 
#MRNs - ENC
#2307280 - 205472336

## Functions

In [198]:
def returnAKIpatients(df, aki_calc_type = 'rolling_window', keep_cols = True):
    
    if aki_calc_type == 'both':
        df = df.groupby('enc', sort=False).apply(lambda d: addRollingWindowAKI(d))
        df = df.reset_index('enc', drop=True).reset_index()
        
        df = df.groupby('mrn', sort=False).apply(lambda d: addBaselineCreat(d))
        df = df.groupby('enc', sort=False).apply(lambda d: addBackCalcAKI(d))
        
    elif aki_calc_type == 'rolling_window':
        df = df.groupby('enc', sort=False).apply(lambda d: addRollingWindowAKI(d))
        
    elif aki_calc_type == 'back_calculate':
        df = df.groupby('mrn', sort=False).apply(lambda d: addBaselineCreat(d))
        df = df.groupby('enc', sort=False).apply(lambda d: addBackCalcAKI(d)) 
    
    return df

def addBaselineCreat(df):
    '''
    Adds the baseline creatinine to a dataframe. The baseline creatinine is defined as the median of the outpatient 
     creatinine values from 365 to 7 days prior to admission.
    
    Input: dataframe (typically of a single patient)
    Output: dataframe with baseline creatinine column added in
    '''
    t_m1y = (df.admission - datetime.timedelta(days=365)).values
    t_m7d = (df.admission - datetime.timedelta(days=7)).values
    
    df['baseline_creat'] = [df[~df.inpatient].set_index('time').sort_index().loc[t_m1y[indx]:t_m7d[indx]].creat.median() for indx in range(df.shape[0])]
    
    return df

def addBackCalcAKI(df):
    '''
    Adds the back-calculated AKI conditions, the KDIGO standards on the outpatient values;
     i.e. a 50% increase from baseline creatinine in <7 days
    
    Input: dataframe (typically of a single encounter)
    Output: dataframe with back-calculated aki values added in
    '''
    backcalc_aki = np.empty(df.shape[0])
    backcalc_aki[:] = np.nan
    
    df = df.sort_values('time')
    df_lf = df.set_index('time').loc[df.admission.values[0]:(df.admission + datetime.timedelta(days=7)).values[0]]
    backcalc_aki[:df_lf.shape[0]] = df_lf.creat > 1.5*df_lf.baseline_creat
    
    df['backcalc_aki'] = backcalc_aki
    
    return df 

def addRollingWindowAKI(df):
    '''
    Adds the AKI conditions based on rolling window definition: 0.3 creat increase in < 48 hrs OR 50% increase in < 7 days
    
    Input: dataframe (typically of a single encounter)
    Output: dataframe with rolling-window aki values added in
    '''
    df = df.set_index('time').sort_index()
    df = df[~df.duplicated()]
    
    df['mincreat_48hr'] = df.creat.rolling(pd.Timedelta('2days'), min_periods=1).min().values
    df['mincreat_7day'] = df.creat.rolling(pd.Timedelta('7days'), min_periods=1).min().values

    df['deltacreat_48hr'] = np.round(df.creat - df.mincreat_48hr, decimals = 3)
    df['deltacreat_7day'] = np.round(df.creat - df.mincreat_7day, decimals = 3)

    df['rollingwindow_aki'] = (df.deltacreat_48hr >= 0.3) | (df.deltacreat_7day > 0.5*df.mincreat_7day)
    
    return df

## Reading in file; managing columns

In [199]:
tmp = out.loc[out.enc == 205472336]
backcalc_aki = np.empty(tmp.shape[0])
backcalc_aki[:] = np.nan
tmp = tmp.sort_values('time')
tmp2 = tmp.set_index('time').sort_index().loc[tmp.admission.values[0]:(tmp.admission + datetime.timedelta(days=7)).values[0]]
backcalc_aki[:tmp2.shape[0]] = tmp2.creat > 1.5*tmp2.baseline_creat
tmp['backcalc_aki'] = backcalc_aki
tmp

AttributeError: 'DataFrame' object has no attribute 'admission'

In [200]:
covid_df = pd.read_csv(r'H:\Data\Standardized AKI definition\dataset\covid creatinines.csv')
covid_df['mrn'] = covid_df.pat_mrn_id.str.strip('MR').astype('int')
covid_df['enc'] = covid_df.enc_id
covid_df['time'] = pd.to_datetime(covid_df.time)
covid_df['sex'] = covid_df.sex.astype('bool')
covid_df['race'] = covid_df.race.astype('bool')
covid_df['inpatient'] = covid_df.inpatient.astype('bool')
covid_df['creat'] = covid_df['creatinine']
covid_df['admission'] = pd.to_datetime(covid_df.admission)
covid_df['discharge'] = pd.to_datetime(covid_df.discharge)
print('Shape:', covid_df.shape)
print(covid_df.dtypes)

Shape: (299718, 14)
pat_mrn_id                object
pat_enc_csn_id             int64
time              datetime64[ns]
creatinine               float64
age                      float64
sex                         bool
race                        bool
enc_id                     int64
admission         datetime64[ns]
discharge         datetime64[ns]
inpatient                   bool
mrn                        int32
enc                        int64
creat                    float64
dtype: object


In [201]:
df = covid_df[['mrn', 'enc', 'time', 'creat', 'age', 'sex', 'race', 'inpatient', 'admission', 'discharge']]

In [202]:
%%time
out = returnAKIpatients(df, aki_calc_type = 'both')

Wall time: 15min 48s


## Adding Baseline Creat & Back-calculated AKI values

In [5]:
%%time
out_rw = returnAKIpatients(df, aki_calc_type = 'rolling_window')

Wall time: 2min 26s


,mrn,enc,time,creat,age,sex,race,inpatient,mincreat_48hr,mincreat_7day,deltacreat_48hr,deltacreat_7day,aki
0,1000026,214990932,2020-03-13 16:20:00,0.50,18.945205,True,False,False,0.50,0.50,0.0,0.00,False
1,1000026,214990932,2020-04-07 20:38:00,0.56,18.945205,True,False,False,0.56,0.56,0.0,0.00,False
2,1000026,214990932,2020-04-10 18:29:00,0.59,18.945205,True,False,True,0.59,0.56,0.0,0.03,False
3,1000026,214990932,2020-04-12 07:46:00,0.56,18.945205,True,False,True,0.56,0.56,0.0,0.00,False
4,1000162,219003501,2020-05-21 23:27:00,0.50,34.052055,True,False,True,0.50,0.50,0.0,0.00,False


In [6]:
%%time
out_bc = returnAKIpatients(df, aki_calc_type = 'back_calculate')

Wall time: 15min 5s


In [153]:
%%time
out = returnAKIpatients(df, aki_calc_type = 'both')

Wall time: 14min 55s


In [160]:
#out.to_csv(r'H:\Data\Standardized AKI definition\dataset\output.csv')

## Two criterion for rolling-window definition of AKI:

#### *$creat \uparrow$ of 0.3 in < 48 hrs* OR *$creat \uparrow$ of 50% in < 7 days*

In [8]:
def eGFR(creat, age, black, female):
    '''
    Calculates the estimated glomerular filtration rate based on the serum creatinine levels, age, sex, and race (black or not black);
    Based on the formula in the paper A New Equation to Estimate Glomerular Filtration Rate (Levey et. Al, 2009) linked below
    
    https://pubmed.ncbi.nlm.nih.gov/19414839/
    
    '''
    #Term 2 - np.clip(creat/(0.9-0.2*female, a_min=1, a_max=None) is the same as taking min(1, creat/k)
    #Term 3 - np.clip(creat/(0.9-0.2*female, a_min=None, a_max=None) is the same as taking max(1, creat/k)
    #where k is the data-derived constant given in the paper: 0.7 for females and 0.9 for males
    
    return 141*(np.clip(creat/(0.9-0.2*female), a_min=1, a_max=None)**(-0.411+0.082*female))*(np.clip(creat/(0.9-0.2*female), a_min=None, a_max=1)**-1.209)*(0.993**age)*(1+female*0.018)*(1+black*0.159)

#Sample test data
creat = np.random.normal(loc=1, scale=0.2, size=10)
age = np.random.normal(loc=55, scale=10, size=10)
black = np.random.rand(10) > 0.5
female = np.random.rand(10) > 0.5

eGFR(creat, age, black, female) 
#values seem pretty reasonable (80-120)

array([ 89.80781217,  86.13568724, 103.64216515, 106.16394424,
       103.53574924, 115.61690503,  90.53939627,  86.40789028,
        88.19213917,  93.91623928])